In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

import requests 
import json
from shutil import which
import time
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from lxml import etree

In [2]:
def scroll(t):
    SCROLL_PAUSE_TIME = t

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [3]:
def extract():
    try:
        price = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "(//section[@class='product-content']//bdi)[last()]")))
        product_details['price'] = price.text
    except:
        try:
            price = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//section[@class='product-content']//bdi/span")))
            product_details['price'] = price.text
            
        except:
            try:
                price = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//section[@class='product-content']//bdi[1]/span/font/font")))
                product_details['price'] = price.text
            except:    
                product_details['price'] = None

    try:
        sku = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//span[@id='sku']")))
        product_details['sku'] = sku.text
    except:
        product_details['sku'] = None

    try:
        avaliability = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[contains(@class,'is-there-stock')]")))
        product_details['available_new'] = avaliability.text
    except:
        product_details['available_new'] = None

    try:
        variant_image = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//img[@id='sg-img']")))
        product_details['variant_image'] = variant_image.get_attribute("src")
    except:
        product_details['variant_image'] = None

In [4]:
def extract_options():
    try:                              
        product_details['waterproof'] = options_waterproof[waterproof]
    except:
        product_details['waterproof'] = None
    try:
        product_details['table_top'] = options_table_top[table_top]
    except:
        product_details['table_top'] = None
    try:
        product_details['fabric'] = options_fabric[fabric]
    except:
        product_details['fabric'] = None
    try:
        product_details['cusion'] = options_cushion[cusion]
    except:
        product_details['cusion'] = None
    try:
        product_details['frame'] = options_frame[frame]
    except:
        product_details['frame'] = None

In [6]:
chrome_options = Options()
headless = True

if headless:
    chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")


prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

# PLease replace your chrome driver path
chrome_path = r"C:\Users\chaitanya\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe"
global driver
driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
driver.get("https://www.skargaarden.com/products/")
scroll(1.5)
time.sleep(1)

product_url = driver.find_elements(By.XPATH, "//ul[@class = 'products columns-4']/li/a")
items = []

for product in product_url:
#     print(product.get_attribute('href'))
    product_details = {}
    
    try:
        name_path = product.find_element(By.XPATH, ".//h2")
        product_details['name'] = name_path.text
    except:
        product_details['name'] = None
        
    if product_details['name'] is not None:
        try:
            category_path = product.find_element(By.XPATH, ".//span[2]")
            product_details['category'] = category_path.text
        except:
            product_details['category'] = None



        try:
            hero_image_path = product.find_element(By.XPATH, ".//div/img")
            hero_image = hero_image_path.get_attribute('src')

            product_details['hero_image'] = hero_image
        except:
            product_details['hero_image'] = None





        href = product.get_attribute('href')
        driver.execute_script("window.open('" +href +"');")
    #     print(href)
        product_details['product_url'] = href
        # switch to new window
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)

        try:
            images = driver.find_elements(By.XPATH, "//figure[@class = 'product-gallery-item']/a/img[@class = 'img']")
            image = []
            for opt in images:
                image.append(opt.get_attribute("src"))
            product_details['images'] = '\n'.join(image)

        except:
            product_details['images'] = None

        try:
            description = driver.find_elements(By.XPATH, "//div[@class = 'product-meta-info-container']/div/div/span")
            tech_description = []
            for opt in description:
                tech_description.append(opt.text)
            product_details['tech description'] = '\n'.join(tech_description)

        except:
            product_details['tech description'] = None
            
        try:
            try:
                product_description = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class = 'product-description']/p/font/font")))
                product_details['product description'] = product_description.text
                
            except:
                product_description = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class = 'product-description']/p")))
                product_details['product description'] = product_description.text
        except:
            product_details['product description'] = None

        try:
            designer = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class = 'product-meta-info-info']/span/a")))
            product_details['designer'] = designer.text
        except:
            product_details['designer'] = None

        try:
            width = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class = 'product-meta-info-info'][1]/div/span[1]")))
            product_details['width'] = width.text
        except:
            product_details['width'] = None

        try:
            height = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class = 'product-meta-info-info'][1]/div/span[3]")))
            product_details['height'] = height.text
        except:
            product_details['height'] = None   

        try:
            length = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class = 'product-meta-info-info'][1]/div/span[2]")))
            product_details['length'] = length.text
        except:
            product_details['length'] = None

        try:
            weight = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class = 'product-meta-info-info'][1]/div/span[4]")))
            product_details['weight'] = weight.text
        except:
            product_details['weight'] = None

        try:
            options = driver.find_elements(By.XPATH, "//select/option")
        except:
            print('No options')

        if len(options) < 1:
            product_details['options'] = 0
            extract()
            extract_options()
            product_details['option(fabric)'] = []
            product_details['option(frame)'] = []
            product_details['option(cushion)'] = []
            product_details['option(waterproof)'] = []
            product_details['option(tabletop)'] = []
            items.append(product_details.copy())
        else:
            product_details['options'] = 1
            try:
                fabric = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//select[contains(@id,'fabric')]/option")))
                options_fabric = []
                for opt in fabric:
                    options_fabric.append(opt.text)
                product_details['option(fabric)'] = options_fabric
            except:
                product_details['option(fabric)'] = []
                
            try:
                frame = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//select[contains(@id,'frame')]/option")))
                options_frame = []
                for opt in frame:
                    options_frame.append(opt.text)
                product_details['option(frame)'] = options_frame
            except:
                product_details['option(frame)'] = []
                
            try:
                cushion = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//select[contains(@id,'cushion')]/option")))
                options_cushion = []
                for opt in cushion:
                    options_cushion.append(opt.text)
                product_details['option(cushion)'] = options_cushion
            except:
                product_details['option(cushion)'] = []
                
            try:
                waterproof = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//select[@id='weatherproof-cover']/option")))
                options_waterproof = []
                for opt in waterproof:
                    options_waterproof.append(opt.text)
                product_details['option(waterproof)'] = options_waterproof
            except:
                product_details['option(waterproof)'] = []
                
            try:
                table_top = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//select[@id='table-top']/option")))
                options_table_top = []
                for opt in table_top:
                    options_table_top.append(opt.text)
                product_details['option(tabletop)'] = options_table_top
            except:
                product_details['option(tabletop)'] = []
                
            options = [product_details['option(frame)'], product_details['option(cushion)'], product_details['option(fabric)'], product_details['option(tabletop)'], product_details['option(waterproof)']]
            options_frame = product_details['option(frame)'] 
            options_cushion = options[1]
            options_fabric = options[2]
            options_table_top = options[3]
            options_waterproof = options[4]
            
            no_of_options = 1
            for opt in options:
                if len(opt)>1:
                    no_of_options *= len(opt) 
            
            product_details['no_of_options'] = no_of_options
            
            if len(options_frame) >1:
                for frame in range(len(options_frame)):
                    select = Select(driver.find_element(By.ID, 'frame'))
                    select.select_by_index(frame)
                    if len(options_cushion) >1:
                        for cusion in range(len(options_cushion)):
                            select = Select(driver.find_element(By.ID, 'cushion'))
                            select.select_by_index(cusion)
                            if len(options_fabric) >1:
                                for fabric in range(len(options_fabric)):
                                    select = Select(driver.find_element(By.XPATH, "//select[contains(@id,'fabric')]"))
                                    select.select_by_index(fabric)
                                    if len(options_table_top) >1:
                                        for table_top in range(len(options_table_top)):
                                            select = Select(driver.find_element(By.ID, 'table-top'))
                                            select.select_by_index(table_top)
                                            if len(options_waterproof) >1:
                                                for waterproof in range(len(options_waterproof)):
                                                    select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                                    select.select_by_index(waterproof)
                                                    extract()
                                                    extract_options()
                                                    
                                                    items.append(product_details.copy())
                                            else:
                                                extract()
                                                extract_options()
                                                items.append(product_details.copy())
                                    elif len(options_waterproof) >1:
                                        for waterproof in range(len(options_waterproof)):
                                            select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                            select.select_by_index(waterproof)
                                            extract()
                                            extract_options()
                                            items.append(product_details.copy())
                                    else:
                                        extract()
                                        extract_options()
                                        items.append(product_details.copy())
                            
                            elif len(options_table_top) >1:
                                for table_top in range(len(options_table_top)):
                                    select = Select(driver.find_element(By.ID, 'table-top'))
                                    select.select_by_index(table_top)
                                    if len(options_waterproof) >1:
                                        for waterproof in range(len(options_waterproof)):
                                            select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                            select.select_by_index(waterproof)
                                            extract()
                                            extract_options()
                                            items.append(product_details.copy())
                                    else:                                        
                                        extract()
                                        extract_options()
                                        items.append(product_details.copy())
                                    
                            elif len(options_waterproof) >1:
                                for waterproof in range(len(options_waterproof)):
                                    select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                    select.select_by_index(waterproof)
                                    extract()
                                    extract_options()
                                    items.append(product_details.copy())
                            else:
                                extract()
                                extract_options()
                                items.append(product_details.copy())
                                
                    elif len(options_fabric) >1:
                        for fabric in range(len(options_fabric)):
                            select = Select(driver.find_element(By.XPATH, "//select[contains(@id,'fabric')]"))
                            select.select_by_index(fabric)
                            if len(options_table_top) >1:
                                for table_top in range(len(options_table_top)):
                                    select = Select(driver.find_element(By.ID, 'table-top'))
                                    select.select_by_index(table_top)
                                    if len(options_waterproof) >1:
                                        for waterproof in range(len(options_waterproof)):
                                            select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                            select.select_by_index(waterproof)
                                            extract()
                                            extract_options()
                                            items.append(product_details.copy())
                                    else:
                                        extract()
                                        extract_options()
                                        items.append(product_details.copy())
                            elif len(options_waterproof) >1:
                                for waterproof in range(len(options_waterproof)):
                                    select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                    select.select_by_index(waterproof)
                                    extract()
                                    extract_options()
                                    items.append(product_details.copy())
                                    
                            else:
                                extract()
                                extract_options()
                                items.append(product_details.copy())

                    elif len(options_table_top) >1:
                        for table_top in range(len(options_table_top)):
                            select = Select(driver.find_element(By.ID, 'table-top'))
                            select.select_by_index(table_top)
                            if len(options_waterproof) >1:
                                for waterproof in range(len(options_waterproof)):
                                    select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                    select.select_by_index(waterproof)
                                    extract()
                                    extract_options()
                                    items.append(product_details.copy())
                            else:
                                extract()
                                extract_options()
                                items.append(product_details.copy())

                    elif len(options_waterproof) >1:
                        for waterproof in range(len(options_waterproof)):
                            select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                            select.select_by_index(waterproof)
                            extract() 
                            extract_options()
                            items.append(product_details.copy())
                    else:
                        extract()
                        extract_options()
                        items.append(product_details.copy())
                                    
            elif len(options_cushion) >1:
                for cusion in range(len(options_cushion)):
                    select = Select(driver.find_element(By.ID, 'cushion'))
                    select.select_by_index(cusion)
                    if len(options_fabric) >1:
                        for fabric in range(len(options_fabric)):
                            select = Select(driver.find_element(By.XPATH, "//select[contains(@id,'fabric')]"))
                            select.select_by_index(fabric)
                            if len(options_table_top) >1:
                                for table_top in range(len(options_table_top)):
                                    select = Select(driver.find_element(By.ID, 'table-top'))
                                    select.select_by_index(table_top)
                                    if len(options_waterproof) >1:
                                        for waterproof in range(len(options_waterproof)):
                                            select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                            select.select_by_index(waterproof)
                                            extract()
                                            extract_options()
                                            items.append(product_details.copy())
                                    else:
                                        extract()
                                        extract_options()
                                        items.append(product_details.copy())
                            elif len(options_waterproof) >1:
                                for waterproof in range(len(options_waterproof)):
                                    select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                    select.select_by_index(waterproof)
                                    extract()
                                    extract_options()
                                    items.append(product_details.copy())
                            else:
                                extract()
                                extract_options()
                                items.append(product_details.copy())

                    elif len(options_table_top) >1:
                        for table_top in range(len(options_table_top)):
                            select = Select(driver.find_element(By.ID, 'table-top'))
                            select.select_by_index(table_top)
                            if len(options_waterproof) >1:
                                for waterproof in range(len(options_waterproof)):
                                    select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                    select.select_by_index(waterproof)
                                    extract()
                                    extract_options()
                                    items.append(product_details.copy())
                            else:                                        
                                extract()
                                extract_options()
                                items.append(product_details.copy())

                    elif len(options_waterproof) >1:
                        for waterproof in range(len(options_waterproof)):
                            select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                            select.select_by_index(waterproof)
                            extract()
                            extract_options()
                            items.append(product_details.copy())

                    else:
                        extract()
                        extract_options()
                        items.append(product_details.copy())

            elif len(options_fabric) >1:
                for fabric in range(len(options_fabric)):
                    select = Select(driver.find_element(By.XPATH, "//select[contains(@id,'fabric')]"))
                    select.select_by_index(fabric)
                    if len(options_table_top) >1:
                        for table_top in range(len(options_table_top)):
                            select = Select(driver.find_element(By.ID, 'table-top'))
                            select.select_by_index(table_top)
                            if len(options_waterproof) >1:
                                for waterproof in range(len(options_waterproof)):
                                    select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                                    select.select_by_index(waterproof)
                                    extract()
                                    extract_options()
                                    items.append(product_details.copy())
                            else:
                                extract()
                                extract_options()
                                items.append(product_details.copy())
                    elif len(options_waterproof) >1:
                        for waterproof in range(len(options_waterproof)):
                            select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                            select.select_by_index(waterproof)
                            extract()
                            extract_options()
                            items.append(product_details.copy())

                    else:
                        extract()
                        extract_options()
                        items.append(product_details.copy())

            elif len(options_table_top) >1:
                for table_top in range(len(options_table_top)):
                    select = Select(driver.find_element(By.ID, 'table-top'))
                    select.select_by_index(table_top)
                    if len(options_waterproof) >1:
                        for waterproof in range(len(options_waterproof)):
                            select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                            select.select_by_index(waterproof)
                            extract()
                            extract_options()
                            items.append(product_details.copy())
                    else:
                        extract()
                        extract_options()
                        items.append(product_details.copy())

            elif len(options_waterproof) >1:
                for waterproof in range(len(options_waterproof)):
                    select = Select(driver.find_element(By.ID, 'weatherproof-cover'))
                    select.select_by_index(waterproof)
                    extract() 
                    extract_options()
                    items.append(product_details.copy())



                                                    
            
                
                
#             print('options')



#         print(product_details)       


#         items.append(product_details.copy())

        driver.close()

        #back to main window
        driver.switch_to.window(driver.window_handles[0])






C:\Users\CHAITA~1\AppData\Local\Temp/ipykernel_10668/1202459736.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)


In [7]:
len(items)

266

In [8]:
items

[{'name': 'ANHOLT LOUNGE CHAIR',
  'category': 'LOUNGE CHAIRS',
  'hero_image': 'https://api.skargaarden.com/api/v1/product-images/ANHLC-DGANHLCC-10152-GC?size=900',
  'product_url': 'https://www.skargaarden.com/products/lounge-chairs/anholt-lounge-chair/',
  'images': 'https://www.skargaarden.com/wp-content/uploads/2020/11/ANHOLT_grop_2-1-289x400.jpg\nhttps://www.skargaarden.com/wp-content/uploads/2020/11/ANHOLT_SOFA_LOUNGE_CLOSE_UP-1-267x400.jpg\nhttps://www.skargaarden.com/wp-content/uploads/2020/11/ANHOLT_SOFA_LOUNGE_CHAIR_TABLE_2-1-322x400.jpg\nhttps://www.skargaarden.com/wp-content/uploads/2020/11/ANHOLT_SOFA_LOUNGE_CHAIR_TABLE-kopiera-1-340x400.jpg\nhttps://www.skargaarden.com/wp-content/uploads/2020/11/ANHOLT_SOFA_LOUNGE_CHAIR_SALTO_LOUNGE_TABLE_3-1-393x400.jpg\nhttps://www.skargaarden.com/wp-content/uploads/2020/11/ANHOLT_grop-1-300x400.jpg\nhttps://www.skargaarden.com/wp-content/uploads/2020/11/ANHOLT_SOFA_LOUNGE_CHAIR_SALTO_LOUNGE_TABLE_2-1-348x400.jpg\nhttps://www.skargaard

In [9]:
import pandas as pd
import numpy as np

df = pd.DataFrame(items, )
df.head()


,name,category,hero_image,product_url,images,tech description,product description,designer,width,height,...,table_top,fabric,cusion,frame,option(fabric),option(frame),option(cushion),option(waterproof),option(tabletop),no_of_options
0,ANHOLT LOUNGE CHAIR,LOUNGE CHAIRS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ch...,https://www.skargaarden.com/wp-content/uploads...,Width: 84 cm ( 33.1 in )\nLength: 80 cm ( 31.5...,The Anholt lounge chair is made of powder coat...,SUSANNE GRØNLUND,Width: 84 cm ( 33.1 in ),Height: 69 cm ( 27.2 in ),...,None,None,None,None,[],[],[],[],[],NaN
1,ANHOLT LOUNGE TABLE,LOUNGE TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ta...,https://www.skargaarden.com/wp-content/uploads...,Width: 60 cm ( 23.6 in )\nLength: 60 cm ( 23.6...,The Anholt lounge table is a lounge table made...,SUSANNE GRØNLUND,Width: 60 cm ( 23.6 in ),Height: 35 cm ( 13.8 in ),...,None,None,None,None,[],[],[],[],[],NaN
2,ANHOLT SIDE TABLE,LOUNGE TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ta...,https://www.skargaarden.com/wp-content/uploads...,Width: 80 cm ( 31.5 in )\nLength: 40 cm ( 15.7...,The Anholt Side Table is a side table made of ...,SUSANNE GRØNLUND,Width: 80 cm ( 31.5 in ),Height: 5 cm ( 2.0 in ),...,None,None,None,None,[],[],[],[],[],NaN
3,ANHOLT SOFA,SOFAS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/sofas/anh...,https://www.skargaarden.com/wp-content/uploads...,Width: 84 cm ( 33.1 in )\nLength: 160 cm ( 63....,The Anholt sofa made of powder coated aluminum...,SUSANNE GRØNLUND,Width: 84 cm ( 33.1 in ),Height: 69 cm ( 27.2 in ),...,None,None,None,None,[],[],[],[],[],NaN
4,ARHOLMA CHAIR,CHAIRS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/chairs/ar...,https://www.skargaarden.com/wp-content/uploads...,Width: 53.5 cm ( 20.9 in )\nLength: 52.5 cm ( ...,The Arholma chair is a minimalistic café chair...,ALEXANDER LERVIK,Width: 53.5 cm ( 20.9 in ),Height: 75.5 cm ( 29.5 in ),...,None,None,No Cushion,None,[],[],"[No Cushion, Silver Grey, Grey Fossil]",[],[],3.0


In [41]:
df.to_csv("skar.csv")

In [10]:
import pandas as pd
import numpy as np

# data = pd.read_csv("skar.csv")
data = df
data

,name,category,hero_image,product_url,images,tech description,product description,designer,width,height,...,table_top,fabric,cusion,frame,option(fabric),option(frame),option(cushion),option(waterproof),option(tabletop),no_of_options
0,ANHOLT LOUNGE CHAIR,LOUNGE CHAIRS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ch...,https://www.skargaarden.com/wp-content/uploads...,Width: 84 cm ( 33.1 in )\nLength: 80 cm ( 31.5...,The Anholt lounge chair is made of powder coat...,SUSANNE GRØNLUND,Width: 84 cm ( 33.1 in ),Height: 69 cm ( 27.2 in ),...,None,None,None,None,[],[],[],[],[],NaN
1,ANHOLT LOUNGE TABLE,LOUNGE TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ta...,https://www.skargaarden.com/wp-content/uploads...,Width: 60 cm ( 23.6 in )\nLength: 60 cm ( 23.6...,The Anholt lounge table is a lounge table made...,SUSANNE GRØNLUND,Width: 60 cm ( 23.6 in ),Height: 35 cm ( 13.8 in ),...,None,None,None,None,[],[],[],[],[],NaN
2,ANHOLT SIDE TABLE,LOUNGE TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ta...,https://www.skargaarden.com/wp-content/uploads...,Width: 80 cm ( 31.5 in )\nLength: 40 cm ( 15.7...,The Anholt Side Table is a side table made of ...,SUSANNE GRØNLUND,Width: 80 cm ( 31.5 in ),Height: 5 cm ( 2.0 in ),...,None,None,None,None,[],[],[],[],[],NaN
3,ANHOLT SOFA,SOFAS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/sofas/anh...,https://www.skargaarden.com/wp-content/uploads...,Width: 84 cm ( 33.1 in )\nLength: 160 cm ( 63....,The Anholt sofa made of powder coated aluminum...,SUSANNE GRØNLUND,Width: 84 cm ( 33.1 in ),Height: 69 cm ( 27.2 in ),...,None,None,None,None,[],[],[],[],[],NaN
4,ARHOLMA CHAIR,CHAIRS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/chairs/ar...,https://www.skargaarden.com/wp-content/uploads...,Width: 53.5 cm ( 20.9 in )\nLength: 52.5 cm ( ...,The Arholma chair is a minimalistic café chair...,ALEXANDER LERVIK,Width: 53.5 cm ( 20.9 in ),Height: 75.5 cm ( 29.5 in ),...,None,None,No Cushion,None,[],[],"[No Cushion, Silver Grey, Grey Fossil]",[],[],3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,VIKEN CHAIR CUSHION,SEAT PILLOWS,https://www.skargaarden.com/wp-content/themes/...,https://www.skargaarden.com/products/seat-pill...,,,None,GUNILLA NORIN,None,None,...,None,None,Heather Chine,None,[],[],"[Sooty, Heather Chine, White, Grey]",[],[],4.0
262,VIKEN CHAIR CUSHION,SEAT PILLOWS,https://www.skargaarden.com/wp-content/themes/...,https://www.skargaarden.com/products/seat-pill...,,,None,GUNILLA NORIN,None,None,...,None,None,White,None,[],[],"[Sooty, Heather Chine, White, Grey]",[],[],4.0
263,VIKEN CHAIR CUSHION,SEAT PILLOWS,https://www.skargaarden.com/wp-content/themes/...,https://www.skargaarden.com/products/seat-pill...,,,None,GUNILLA NORIN,None,None,...,None,None,Grey,None,[],[],"[Sooty, Heather Chine, White, Grey]",[],[],4.0
264,VIKEN TABLE LARGE,TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/tables/vi...,https://www.skargaarden.com/wp-content/uploads...,Width: 121 cm ( 47.6 in )\nLength: 140 cm ( 55...,The Viken table is made from teak and is folda...,GUNILLA NORIN,Width: 121 cm ( 47.6 in ),Height: 72.5 cm ( 28.3 in ),...,None,None,Grey,None,[],[],[],[],[],NaN


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 266 non-null    object 
 1   category             266 non-null    object 
 2   hero_image           266 non-null    object 
 3   product_url          266 non-null    object 
 4   images               266 non-null    object 
 5   tech description     266 non-null    object 
 6   product description  219 non-null    object 
 7   designer             256 non-null    object 
 8   width                244 non-null    object 
 9   height               241 non-null    object 
 10  length               241 non-null    object 
 11  weight               241 non-null    object 
 12  options              266 non-null    int64  
 13  price                266 non-null    object 
 14  sku                  266 non-null    object 
 15  available_new        263 non-null    obj

In [12]:
data.drop(['option(fabric)', 'option(frame)', 'option(cushion)', 'option(waterproof)', 'option(tabletop)' ], axis=1, inplace=True)


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 266 non-null    object 
 1   category             266 non-null    object 
 2   hero_image           266 non-null    object 
 3   product_url          266 non-null    object 
 4   images               266 non-null    object 
 5   tech description     266 non-null    object 
 6   product description  219 non-null    object 
 7   designer             256 non-null    object 
 8   width                244 non-null    object 
 9   height               241 non-null    object 
 10  length               241 non-null    object 
 11  weight               241 non-null    object 
 12  options              266 non-null    int64  
 13  price                266 non-null    object 
 14  sku                  266 non-null    object 
 15  available_new        263 non-null    obj

In [17]:
data.options

0      0
1      0
2      0
3      0
4      1
      ..
261    1
262    1
263    1
264    0
265    0
Name: options, Length: 266, dtype: int64

In [14]:
data.rename(columns={'waterproof': 'option(waterproof)', 'table_top': 'option(tabletop)', \
                  'fabric':'option(fabric)', 'frame': 'option(frame)', 'cusion':'option(cushion)' }, inplace=True)


In [15]:
data.head()

,name,category,hero_image,product_url,images,tech description,product description,designer,width,height,...,price,sku,available_new,variant_image,option(waterproof),option(tabletop),option(fabric),option(cushion),option(frame),no_of_options
0,ANHOLT LOUNGE CHAIR,LOUNGE CHAIRS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ch...,https://www.skargaarden.com/wp-content/uploads...,Width: 84 cm ( 33.1 in )\nLength: 80 cm ( 31.5...,The Anholt lounge chair is made of powder coat...,SUSANNE GRØNLUND,Width: 84 cm ( 33.1 in ),Height: 69 cm ( 27.2 in ),...,"$3,000",ANHLC-DGANHLCC-10152-GC,In stock,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,None,None,NaN
1,ANHOLT LOUNGE TABLE,LOUNGE TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ta...,https://www.skargaarden.com/wp-content/uploads...,Width: 60 cm ( 23.6 in )\nLength: 60 cm ( 23.6...,The Anholt lounge table is a lounge table made...,SUSANNE GRØNLUND,Width: 60 cm ( 23.6 in ),Height: 35 cm ( 13.8 in ),...,$880,ANHLT-DG,Out of stock\nDelivery within 4-6 weeks.\nThe ...,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,None,None,NaN
2,ANHOLT SIDE TABLE,LOUNGE TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ta...,https://www.skargaarden.com/wp-content/uploads...,Width: 80 cm ( 31.5 in )\nLength: 40 cm ( 15.7...,The Anholt Side Table is a side table made of ...,SUSANNE GRØNLUND,Width: 80 cm ( 31.5 in ),Height: 5 cm ( 2.0 in ),...,$600,ANHST-DG,In stock,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,None,None,NaN
3,ANHOLT SOFA,SOFAS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/sofas/anh...,https://www.skargaarden.com/wp-content/uploads...,Width: 84 cm ( 33.1 in )\nLength: 160 cm ( 63....,The Anholt sofa made of powder coated aluminum...,SUSANNE GRØNLUND,Width: 84 cm ( 33.1 in ),Height: 69 cm ( 27.2 in ),...,"$5,500",ANHS-DGANHSC-10152-GC,In stock,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,None,None,NaN
4,ARHOLMA CHAIR,CHAIRS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/chairs/ar...,https://www.skargaarden.com/wp-content/uploads...,Width: 53.5 cm ( 20.9 in )\nLength: 52.5 cm ( ...,The Arholma chair is a minimalistic café chair...,ALEXANDER LERVIK,Width: 53.5 cm ( 20.9 in ),Height: 75.5 cm ( 29.5 in ),...,$540,ARHC-DG,In stock,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,No Cushion,None,3.0


In [18]:
data.drop([ 'no_of_options', 'options' ], axis=1, inplace=True)


In [19]:
data

,name,category,hero_image,product_url,images,tech description,product description,designer,width,height,...,weight,price,sku,available_new,variant_image,option(waterproof),option(tabletop),option(fabric),option(cushion),option(frame)
0,ANHOLT LOUNGE CHAIR,LOUNGE CHAIRS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ch...,https://www.skargaarden.com/wp-content/uploads...,Width: 84 cm ( 33.1 in )\nLength: 80 cm ( 31.5...,The Anholt lounge chair is made of powder coat...,SUSANNE GRØNLUND,Width: 84 cm ( 33.1 in ),Height: 69 cm ( 27.2 in ),...,Weight: 16 kg ( 35.3 lb ),"$3,000",ANHLC-DGANHLCC-10152-GC,In stock,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,None,None
1,ANHOLT LOUNGE TABLE,LOUNGE TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ta...,https://www.skargaarden.com/wp-content/uploads...,Width: 60 cm ( 23.6 in )\nLength: 60 cm ( 23.6...,The Anholt lounge table is a lounge table made...,SUSANNE GRØNLUND,Width: 60 cm ( 23.6 in ),Height: 35 cm ( 13.8 in ),...,Weight: 5.5 kg ( 11.0 lb ),$880,ANHLT-DG,Out of stock\nDelivery within 4-6 weeks.\nThe ...,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,None,None
2,ANHOLT SIDE TABLE,LOUNGE TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/lounge-ta...,https://www.skargaarden.com/wp-content/uploads...,Width: 80 cm ( 31.5 in )\nLength: 40 cm ( 15.7...,The Anholt Side Table is a side table made of ...,SUSANNE GRØNLUND,Width: 80 cm ( 31.5 in ),Height: 5 cm ( 2.0 in ),...,Weight: 2.5 kg ( 4.4 lb ),$600,ANHST-DG,In stock,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,None,None
3,ANHOLT SOFA,SOFAS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/sofas/anh...,https://www.skargaarden.com/wp-content/uploads...,Width: 84 cm ( 33.1 in )\nLength: 160 cm ( 63....,The Anholt sofa made of powder coated aluminum...,SUSANNE GRØNLUND,Width: 84 cm ( 33.1 in ),Height: 69 cm ( 27.2 in ),...,Weight: 32 kg ( 70.5 lb ),"$5,500",ANHS-DGANHSC-10152-GC,In stock,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,None,None
4,ARHOLMA CHAIR,CHAIRS,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/chairs/ar...,https://www.skargaarden.com/wp-content/uploads...,Width: 53.5 cm ( 20.9 in )\nLength: 52.5 cm ( ...,The Arholma chair is a minimalistic café chair...,ALEXANDER LERVIK,Width: 53.5 cm ( 20.9 in ),Height: 75.5 cm ( 29.5 in ),...,Weight: 5.5 kg ( 11.0 lb ),$540,ARHC-DG,In stock,https://api.skargaarden.com/api/v1/product-ima...,None,None,None,No Cushion,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,VIKEN CHAIR CUSHION,SEAT PILLOWS,https://www.skargaarden.com/wp-content/themes/...,https://www.skargaarden.com/products/seat-pill...,,,None,GUNILLA NORIN,None,None,...,None,$,VIKC-SUN-NN,In stock,https://www.skargaarden.com/wp-content/themes/...,None,None,None,Heather Chine,None
262,VIKEN CHAIR CUSHION,SEAT PILLOWS,https://www.skargaarden.com/wp-content/themes/...,https://www.skargaarden.com/products/seat-pill...,,,None,GUNILLA NORIN,None,None,...,None,$,VIKC-SUN-WN,In stock,https://www.skargaarden.com/wp-content/themes/...,None,None,None,White,None
263,VIKEN CHAIR CUSHION,SEAT PILLOWS,https://www.skargaarden.com/wp-content/themes/...,https://www.skargaarden.com/products/seat-pill...,,,None,GUNILLA NORIN,None,None,...,None,$,VIKC-SUN-3659-GA,In stock,https://www.skargaarden.com/wp-content/themes/...,None,None,None,Grey,None
264,VIKEN TABLE LARGE,TABLES,https://api.skargaarden.com/api/v1/product-ima...,https://www.skargaarden.com/products/tables/vi...,https://www.skargaarden.com/wp-content/uploads...,Width: 121 cm ( 47.6 in )\nLength: 140 cm ( 55...,The Viken table is made from teak and is folda...,GUNILLA NORIN,Width: 121 cm ( 47.6 in ),Height: 72.5 cm ( 28.3 in ),...,Weight: 17 kg ( 37.5 l

In [20]:
df_full = data.reindex(columns=[*data.columns.tolist(), 'GTIN', 'option(size)', 'option(cage)', \
                             'option(color)' ,'option(canopy color)', 'color','display_dimensions',\
                              'option(finish)', 'finish','display_dimensions', 'depth', \
                    'diameter', 'arm height', 'seat height', 'table clearance height', \
                    'maximum dimensions', 'files','made_in', 'canopy dimensions', 'installations',\
                              'collection' , 'voltage', 'bulb_type', 'bulb_text', 'UL Listed', \
                    'certifications', 'warranty & care', 'position', 'notes'], fill_value= None)

In [21]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    266 non-null    object 
 1   category                266 non-null    object 
 2   hero_image              266 non-null    object 
 3   product_url             266 non-null    object 
 4   images                  266 non-null    object 
 5   tech description        266 non-null    object 
 6   product description     219 non-null    object 
 7   designer                256 non-null    object 
 8   width                   244 non-null    object 
 9   height                  241 non-null    object 
 10  length                  241 non-null    object 
 11  weight                  241 non-null    object 
 12  price                   266 non-null    object 
 13  sku                     266 non-null    object 
 14  available_new           263 non-null    ob

In [22]:
df_full.to_excel('skargaarden_new.xlsx')